# Movie Recommendations via Matrix Factorization

## Introduction
This notebook explains the main ideas behind a common class of algorithms for building recommender systems.
Assume we have a data set of triples (user, item, rating). 
The main idea is to represent this data as a matrix $X$, where the entry in the $i$th row and $j$th column, $X_{ij}$,
is the rating that user $i$ gave to item $j$.
It is almost never the case that every user has rated every item, so $X$ has many missing values.
The goal is to learn how to fill in the missing values.
In other words, we want to figure out which items a user hasn't rated but would rate highly, so that we can recommend these items to them.

We do this by finding matrices $P$ and $Q$ such that $X \approx PQ$.
Why would this be useful? 
The trick is that both $P$ and $Q$ are supposed to be "low rank."
That means, if $X$ is an $m \times n$ matrix, then $P$ is $m \times k$ and $Q$ is $k \times n$ 
for some $k$ typically much smaller than either $m$ or $n$.
Thus, we are trying to capture the patterns present in (up to) $mn$ numbers, using only $k(m + n)$ parameters.
Provided $k$ is small enough, there is clearly some kind of compression or distillation of information,
so it makes sense (to me, anyway) to claim that we've learned something about $X$.
We can then fill in the missing values of $X$ with the corresponding values of $PQ$.
As for exactly what we've learned, I'll talk about below.

## Intuition

The matrix factorization procedure outlined above actually has a nifty geometric interpretation.
Suppose, as we mentioned above, that $X$ is an $m \times n$ matrix.
That means there are $m$ users and $n$ items.
Since $P$ is $m \times k$, $P$ has a row for every user.
Similarly, since $Q$ is $k \times n$, $Q$ has a column for every item.
So, every user and every item gets assigned a $k$-dimensional vector, corresponding to a row or column of $P$ or $Q$ respectively.
Thus, when we factorize $X$, we're learning vector representations of our users and items in a $k$-dimensional latent space.

Now, letting $p_i$ denote row $i$ of $P$, and letting $q_j$ denote column $j$ of $Q$,
$X_{ij} \approx [PQ]_{ij} = p_i \cdot q_j$.
This means that our estimated rating for user $i$ on item $j$ is simply the dot product of their two vector representations.
Now, you may remember that the dot product between two vectors is the product of their lengths times the cosine of the angle between them.
Thus, ignoring vector length for a moment, if the vectors for a given user and item point in generally the same direction,
then we predict that the user will rate that item highly.
Items that are usually rated highly will have longer vector representations.
Similarly, users that tend to rate items more highly will have longer vector representations.

So factoring the matrix learns a representation with natural geometric properties.
Neat.


## Mathematical Details

Ok, now we're ready for the nitty gritty. 

The formal statement of the problem is to find $P$ and $Q$ minimizing 
$$
L(X, P, Q, \lambda) := \sum_{i,j} (X_{ij} - [PQ]_{ij}) ^ 2 + \lambda (||P||^2 + ||Q||^2),
$$
where the sum is over only the observed entries of $X$, and $\lambda$ is a hyperparameter controlling the amount of $L_2$ regularization.

We can optimize this loss function via gradient descent, deep learning style. So, let's figure out the gradient.
(I skipped some steps. You can either take my word for it, or grab a piece of paper. It's not too bad.)

\begin{align}
\frac{\partial L}{\partial P_{il}} 
& = \sum_j \frac{\partial}{\partial P_{il}} (X_{ij} - [PQ]_{ij}) ^ 2 + \frac{\partial}{\partial P_{il}} \lambda ||P||^2 \\
& = -\sum_j 2 (X_{ij} - [PQ]_{ij}) Q_{lj} + 2 \lambda P_{il}
\end{align}

where the sum is over all the $j$ where the $ij$ entry of $X$ is known.

Similarly,
$$
\frac{\partial L}{\partial Q_{lj}} = -\sum_i 2 (X_{ij} - [PQ]_{ij}) P_{il} + 2 \lambda Q_{lj},
$$

where the sum is over all the $i$ where the $ij$ entry of $X$ is known.

And that's really all we need to perform gradient descent.

## Example Implementation

So, now that we've discussed the mathematical details, we have all the information we need to actually implement a matrix factorization algorithm.
I do this in NumPy, below. My implementation is not terribly efficient or rigorously tested. 
But I'm not trying to show best practices here. I'm just trying to show a sketch the theoretical ideas in action.

In [1]:
import numpy as np
import pandas as pd

Since $X$ will generally be very sparse, and we need a way to keep track of which entries are missing, I'll simply represent $X$ as a dictionary
where the keys are $(i, j)$ pairs, and the values are the $X_{ij}$ entries. The absence of a key denotes that the corresponding entry is not known.

Since $P$ and $Q$ will be non-sparse, we can just use ordinary numpy arrays for those, but we'll transpose $Q$, so that way
there is symmetry in our handling of the two. 
In particular, since NumPy lets us index rows like `P[i]`, making the rows the import vectors for both $P$ and $Q$ makes the indexing a little nicer to work with.

First, we define the loss function. That first line is kind of a doozy. I'm using the `zip(*x)` trick to unzip or transpose the list of tuples. This turns the list of 3-tuples into 3 very long tuples. But then I have to convert those tuples into lists for the indexing to work properly.

In [2]:
def loss(X, P, Q, l2):
    i, j, x = (list(a) for a in zip(*[(i, j, x) for (i, j), x in X.items()]))
    Xhat = np.sum(P[i] * Q[j], axis=1) # row-wise dot product
    return np.sum((x - Xhat) ** 2) + l2 * (np.sum(P ** 2) + np.sum(Q ** 2))

Now I'm just going to do a little sanity check.

In [3]:
from itertools import product
P = np.random.randn(100, 3)
Q = np.random.randn(200, 3)
PQ = P @ Q.T
X = {(i, j): PQ[i, j] for i, j in product (range(100), range(200))}
print(loss(X, P, Q, 0))
print(np.sum((PQ - P@Q.T) ** 2))
P = 2 * P
print(loss(X, P, Q, 0))
print(np.sum((PQ - P@Q.T) ** 2))
P = P + 1
Q = Q + 1
print(loss(X, P, Q, 0))
print(np.sum((PQ - P@Q.T) ** 2))

4.121366454959534e-28
0.0
62301.46837253707
62301.46837253707
568809.3737568273
568809.3737568273


The loss function looks reasonable. Great!

Next, we implement the (analytic) gradient, as well as a slow numerical gradient to check our work.

In [4]:
def getj(X, i):
    js = [j for (k, j) in X.keys() if k == i]
    return np.array(js)

def geti(X, j):
    iis = [i for (i, k) in X.keys() if k == j]
    return np.array(iis)

def grad(X, P, Q, l2):
    dP = 2 * l2 * P
    for i in range(P.shape[0]):
        js = getj(X, i)
        for j in js:
            dP[i] += -2 * Q[j] * (X[(i, j)] - P[i].dot(Q[j]))
    dQ = 2 * l2 * Q
    for j, row in enumerate(Q):
        iis = geti(X, j)
        for i in iis:
            dQ[j] += -2 * P[i] * (X[(i, j)] - P[i].dot(Q[j]))
    return dP, dQ

In [5]:
def num_grad(X, P, Q, l2, eps=1e-5):
    dP = np.zeros(P.shape)
    dQ = np.zeros(Q.shape)
    for i in range(P.shape[0]):
        for j in range(P.shape[1]):
            P[i, j] += eps
            pos = loss(X, P, Q, l2)
            P[i, j] -= 2 * eps
            neg = loss(X, P, Q, l2)
            P[i, j] += eps
            dP[i, j] = (pos - neg) / (2 * eps)
    for i in range(Q.shape[0]):
        for j in range(Q.shape[1]):
            Q[i, j] += eps
            pos = loss(X, P, Q, l2)
            Q[i, j] -= 2 * eps
            neg = loss(X, P, Q, l2)
            Q[i, j] += eps
            dQ[i, j] = (pos - neg) / (2 * eps)
    return dP, dQ
            

Below, we perform a little random gradient check. I just ran the cell a few times and make sure the two numbers it outputs are always nearly zero.

In [6]:
P = np.random.randn(4, 2)
Q = np.random.randn(8, 2)
PQ = P @ Q.T
X = {}
for i in range(PQ.shape[0]):
    for j in range(PQ.shape[1]):
        if (i + j) % 2 == 0:
            X[(i, j)] = PQ[i, j]
l2 = 0.2
P = np.random.randn(4, 2)
Q = np.random.randn(8, 2)
dPn, dQn = num_grad(X, P, Q, l2)
dPa, dQa = grad(X, P, Q, l2)
print(np.sum((dPa - dPn)** 2))
print(np.sum((dQa - dQn) ** 2))

3.6341080462781927e-19
1.9024847488473296e-19


It looks like we've passed the gradient check! 

Now we just need a little bit more code to implement minibatch gradient descent.

In [7]:
# get a random sample of X, for minibatch sgd
from random import sample
def make_batch(X, size):
    return {key: X[key] for key in sample(list(X), size)}

In [8]:
def fit(X, k, l2, batch_size, step_size, epochs, P=None, Q=None, Xval=None, dims=None):
    """Factorize the matrix X using a rank k approximation.
    
    X - the matrix to factor, in sparse dictionary form
    k - the rank of approximation
    l2 - controls the l2 regularization amount
    batch_size - the batch size for minibatch sgd
    step_size - the step size for minibatch sgd
    epochs - the number of epochs to run for,
             where an epoch is defined to be
             len(X) / batch_size many batches
    P, Q - Initializations. If None, initialize randomly
    Xval - held out data to compare performance on.
    dims - (m, n) the dimensions of the implied matrix X
           if none, the functions tries to infer
    """
    if dims is None:
        pshape = max(i for i, j in X.keys()) + 1
        qshape = max(j for i, j in X.keys()) + 1
    else:
        pshape, qshape = dims
    if P is None:
        P = np.random.randn(pshape, k)
    if Q is None:
        Q = np.random.randn(qshape, k)
    epoch_length = int(len(X) / batch_size) + 1
    print(f"Training for {epochs} epochs, each of {epoch_length} batches, each of size {batch_size}.")
    print(f"Epoch -1: Training Loss = {loss(X, P, Q, l2)/len(X)}, Validation Loss = {loss(Xval, P, Q, 0)/len(Xval)}")
    for i in range(epochs):
        for j in range(epoch_length):
            print(f"\r========== Epoch {i} in progress...{j * 100 // epoch_length}% ==========", end="")
            batch = make_batch(X, batch_size)
            dP, dQ = grad(batch, P, Q, l2)
            P -= step_size * dP
            Q -= step_size * dQ
        print(f"\r========== Epoch {i} in progress...100% ==========")
        if Xval is None:
            print(f"Epoch {i}: Loss = {loss(X, P, Q, l2)}")
        else:
            # never use l2 penalty in validation loss
            # normalize to be on same scale
            print(f"Epoch {i}: Training Loss = {loss(X, P, Q, l2)/len(X)}, Validation Loss = {loss(Xval, P, Q, 0)/len(Xval)}")
    return P, Q

Let's check that our code works on a tiny synthetic data set.

In [9]:
from random import shuffle
P = np.random.randn(100, 3)
Q = np.random.randn(200, 3)
PQ = P @ Q.T
X = {(i, j): PQ[i, j] for i, j in product(range(100), range(200))}
cutoff = int(len(X) * .8)
keys = list(X.keys())
shuffle(keys)
Xtrain = {key: X[key] for key in keys[:cutoff]}
Xval = {key: X[key] for key in keys[cutoff:]}
print(loss(X, P, Q, 0))

3.608090487723571e-28


In [10]:
P, Q = fit(Xtrain, Xval=Xval, k=3, l2=0.01, batch_size=100, step_size=0.01, epochs=10, dims=(100, 200))

Training for 10 epochs, each of 161 batches, each of size 100.
Epoch -1: Training Loss = 5.894358217777715, Validation Loss = 6.094976374192005
========== Epoch 0 in progress...100% ==========
Epoch 0: Training Loss = 2.5171315979343523, Validation Loss = 2.781513141416761
========== Epoch 1 in progress...100% ==========
Epoch 1: Training Loss = 0.8978084382898048, Validation Loss = 1.0669522602969912
========== Epoch 2 in progress...100% ==========
Epoch 2: Training Loss = 0.10486428181981564, Validation Loss = 0.1297978833530837
========== Epoch 3 in progress...100% ==========
Epoch 3: Training Loss = 0.0063031305698191195, Validation Loss = 0.008025445259185008
========== Epoch 4 in progress...100% ==========
Epoch 4: Training Loss = 0.0014566148028996835, Validation Loss = 0.0013003985403829282
========== Epoch 5 in progress...100% ==========
Epoch 5: Training Loss = 0.0011699102320052077, Validation Loss = 0.0008398687124477883
========== Epoch 6 in progress...100% ==========
Epoc

Okay, great. The model has quickly and sucessfully driven both the training and validation losses close to zero.
Thus, the code is most likely working correctly.

Since everything seems to be in order, we're finally ready to try out the code on a real data set.
I'm going to use the small MovieLens data set (`ml-latest-small`), available freely from GroupLens [here](https://grouplens.org/datasets/movielens/).
This small data set consists of 100,000 ratings applied to 9,000 movies by 600 users.

First, let's load the data and take a peek at it.

In [11]:
df = pd.read_csv("ml-latest-small/ratings.csv")
print(df.shape)
df.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Now, let's process this data frame into the dictionary format required by the code.

In [12]:
# first step: process data into sparse matrix dictionary form
# for that I need to re-index the userid and movieid to be consecutive non-negative integers
user_ids = sorted(list(set(df.userId)))
movie_ids = sorted(list(set(df.movieId)))
user_index_map = {key: val for val, key in enumerate(user_ids)}
movie_index_map = {key: val for val, key in enumerate(movie_ids)}
df['user_index'] = df.userId.map(user_index_map)
df['movie_index'] = df.movieId.map(movie_index_map)
df.head()

,userId,movieId,rating,timestamp,user_index,movie_index
0,1,1,4.0,964982703,0,0
1,1,3,4.0,964981247,0,2
2,1,6,4.0,964982224,0,5
3,1,47,5.0,964983815,0,43
4,1,50,5.0,964982931,0,46


In [13]:
# build dictionary matrix
keys = zip(df.user_index, df.movie_index)
vals = df.rating
X = {(i, j): x for (i, j), x in zip(keys, vals)}

# split data
cutoff = int(len(X) * .8)
keys = list(X)
shuffle(keys)
Xtrain = {key: X[key] for key in keys[:cutoff]}
Xval = {key: X[key] for key in keys[cutoff:]}
dims = df.user_index.max() + 1, df.movie_index.max() + 1

Okay, now we're ready to fit the model. To do so, we'd run the follwing code:
```
P, Q = fit(Xtrain, Xval=Xval, k=200, dims=dims, l2=0.01, batch_size=100, step_size=0.001, epochs=a_really_big_number)
```
I've already done this elsewhere, since it takes forever to run, and I want this notebook to execute quickly.

So I'll just load up the pretrained $P$ and $Q$ matrices. We have a pretty low MSE on the validation data. Nice.

In [14]:
P, Q = np.load('P.npy'), np.load('Q.npy')
loss(Xval, P, Q, 0) / len(Xval)

0.5958826178605978

Let's look at some individual predictions. Below, we can see that the predicted ratings are pretty good. They are generally within 1 star (the rating unit) of the true rating. Though sometimes they can be quite far off.

In [15]:
# This makes a table of sorts of the real and predicted ratings for the val set
[(user, movie, actual, P[user].dot(Q[movie])) for ((user, movie), actual) in Xval.items()][:20]

[(451, 275, 5.0, 4.644598369878881),
 (312, 559, 1.0, 1.5419352060574192),
 (336, 23, 5.0, 3.334300660089183),
 (327, 7061, 4.0, 3.4669705536037108),
 (447, 3490, 3.0, 3.0905088672890457),
 (584, 1839, 4.0, 2.76536382219863),
 (364, 5363, 2.0, 1.9991478770895572),
 (533, 7823, 2.0, 2.244419421995681),
 (606, 1157, 4.0, 3.5923271202926546),
 (447, 8022, 3.0, 1.2447174489819972),
 (609, 7123, 3.0, 2.8549683445071605),
 (605, 2226, 3.0, 2.605996177558488),
 (602, 2312, 4.0, 3.823142108372504),
 (359, 1198, 2.0, 2.7603552309162906),
 (135, 540, 4.0, 2.890747829885643),
 (411, 398, 4.0, 4.399439156786133),
 (90, 1043, 3.5, 3.5071153854308044),
 (540, 254, 3.0, 3.56781811791424),
 (436, 442, 3.0, 2.924777167945934),
 (67, 1570, 3.0, 3.0071861743043464)]

Okay, this is cool.
But how do we generate predictions for new users? Well, one way is to add the user to the data and retrain the whole model from scratch.
Clearly, that is not going to be very efficient. Of course, we could try to reuse the existing $P$ and $Q$ to give a good initalization
(a warm-start, as it is sometimes called) to the 
new training run. Taking this idea to an extreme, we could leave $P$ and $Q$ as completely fixed as possible, and only try to learn an approximate new
latent vector for the new user. This new vector, call it $p$, is supposed to satisfy $x \approx pQ$, where $x$ is the row of ratings for the new user,
and $Q$ is the already-learned representation of the items. So, in other words, we want to minimize (wrt $p$ only)

$$
||Q^Tp^T - x^T||^2 + \lambda ||p|| ^ 2
$$

But this is just ridge regression(!), where $Q^T = X$, $p^T = \beta$, and $x^T = y$. So now we have an easy way to generate predictions for a hypothetical new user. For example, we could have a guest user input ratings for a few movies, and then we could output movie recommendations in real time, without having to wait for the model to retrain.

If we wanted to make this idea really efficient, I'm sure we could cache a matrix decomposition (or similarly a matrix inverse, but something like the Cholesky factorization usually has better numerical properties) so that we only have to do a few matrix multiplications to get the recommendations. For our purposes, I'll just use scikit-learn's implementation of ridge regression.

In [16]:
# load up the movie info, set the index to movieId (for fast lookup)
# and then test out a look up (should give Persuasion)
movie_df = pd.read_csv('ml-latest-small/movies.csv')
movie_df = movie_df.set_index('movieId')
movie_df.loc[28].title

'Persuasion (1995)'

In [17]:
# Get the ids for all the children's movies
children_ids = movie_df[movie_df.genres.str.contains("Children")].index.values
# Get the ids for all the dramas
drama_ids = movie_df[movie_df.genres.str.contains("Drama")].index.values

In [18]:
from sklearn.linear_model import Ridge

# Let's suppose our hypothetical new user has given 5 star ratings to the 10 children's movies
ratings = {movie_index_map[key]: 5.0 for key in children_ids[:10]}

# Get the rows of Q for the corresponding movies
Q_filter = Q[list(ratings.keys())]
x = np.array(list(ratings.values()))

# Fit the ridge model and get the ratings predictions out
full_ratings = Ridge(alpha=1, fit_intercept=False).fit(Q_filter, x).predict(Q)
# associate the title to each rating
full_ratings = [(movie_df.loc[movie_ids[i]].title, rating) for i, rating in enumerate(full_ratings)]
# Sort by highest rating to lowest
full_ratings.sort(key=lambda x: x[1], reverse=True)

# Print the top 20 recommendations
full_ratings[:20]

[('Forrest Gump (1994)', 6.383852472927765),
 ('Terminator 2: Judgment Day (1991)', 6.2847574267970145),
 ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  6.275125559269302),
 ('Fugitive, The (1993)', 6.228253213211051),
 ('Shrek (2001)', 6.171320227058867),
 ('Shawshank Redemption, The (1994)', 6.17112826012575),
 ('Star Wars: Episode IV - A New Hope (1977)', 6.1668186290012805),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 6.14994676458946),
 ('Hoop Dreams (1994)', 6.1385696988648135),
 ('Postman, The (Postino, Il) (1994)', 6.04826334311373),
 ('Speed (1994)', 5.986584934056694),
 ('Princess Bride, The (1987)', 5.979179875450452),
 ('Lawrence of Arabia (1962)', 5.949116095984733),
 ('Braveheart (1995)', 5.94549892748534),
 ('Lion King, The (1994)', 5.942644653553394),
 ('Good Will Hunting (1997)', 5.92669912205136),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 5.923137999597239),
 ('Terminator, The (1984)', 5.917200585414389

In [19]:
# Let's do it again with someone who likes dramas

# Now the hypothetical new user has given 5 star ratings to the 10 dramas
ratings = {movie_index_map[key]: 5.0 for key in drama_ids[:10]}

# Get the rows of Q for the corresponding movies
Q_filter = Q[list(ratings.keys())]
x = np.array(list(ratings.values()))

# Fit the ridge model and get the ratings predictions out
full_ratings = Ridge(alpha=1, fit_intercept=False).fit(Q_filter, x).predict(Q)
# associate the title to each rating
full_ratings = [(movie_df.loc[movie_ids[i]].title, rating) for i, rating in enumerate(full_ratings)]
# Sort by highest rating to lowest
full_ratings.sort(key=lambda x: x[1], reverse=True)

# Print the top 20 recommendations
full_ratings[:20]

[('Forrest Gump (1994)', 6.484102530807535),
 ('Shawshank Redemption, The (1994)', 6.296902511472191),
 ("Schindler's List (1993)", 6.117397319506058),
 ('Usual Suspects, The (1995)', 6.072240703293817),
 ('Postman, The (Postino, Il) (1994)', 6.020337246777842),
 ('Braveheart (1995)', 5.953058316887174),
 ('Silence of the Lambs, The (1991)', 5.868094031620335),
 ('Hoop Dreams (1994)', 5.781996588356677),
 ('Shrek (2001)', 5.773151324852257),
 ('American History X (1998)', 5.745147853064867),
 ('Lion King, The (1994)', 5.718513660705401),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 5.71546782704302),
 ('Matrix, The (1999)', 5.697166598602758),
 ('Dances with Wolves (1990)', 5.696776595603996),
 ('Fugitive, The (1993)', 5.681671806559443),
 ('Terminator 2: Judgment Day (1991)', 5.67062190066647),
 ('Green Mile, The (1999)', 5.6598480424153985),
 ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  5.656563000976246),
 ('Saving Private Ryan (1998

You probably noticed that the recommendations are pretty weird.
Even though we had our hypothetical user rated a bunch of children's movies highly, our recommender system
seems to just output an arbitrary assortment of highly acclaimed films.
And a similar thing happens for the person who loves dramas.

The problem is that some movies are universally acclaimed, according to the data.
So the model learns give these movies long latent vectors, which means that everyone ends up rating these movies highly, regardless of their own
particular preferences. 

There are modifications we can make to the algorithm to help account for this problem.
For example, we can add an additional parameter for each movie that accounts for the average rating of the movie.
Then the dot product of the user and movie vectors accounts for a deviation from the usual rating.
We can extend the same idea to the users. Since some users are more likely to give higher ratings overall, and some are more likely
to give lower ratings, accounting for the average behavior of the user can also help isolate the interaction between user and movie.
Once we've separated out the interaction from the average behavior of the users and movies, then we can adjust our recommendations
accordingly.

I won't be implementing these improvements in my code right now, since I think this tutorial is long enough.
If you want to see how these modified algorithms perform, or if you want to create a recommender system to use in practice,
I'd recommend the [Surprise scikit.](http://surpriselib.com)

Well, our recommendations didn't turn out to be amazing, but hopefully if you read this all then you've learned that there's really nothing
too complicated about the core ideas behind matrix factorization recommender systems. Thanks for reading.